In [1]:
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import models
import multiprocessing

In [2]:
df = pd.read_csv("../data/train.csv")
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)
train_df.shape, val_df.shape

((8325, 4), (2082, 4))

In [3]:
class PaddyDataset(Dataset):
    def __init__(self, df, img_dir, transformer=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transformer
        
        classes = self.df['label'].unique()
        self.label2idx = {label: idx for idx, label in enumerate(classes)}
        self.idx2label = {idx: label for idx, label in enumerate(classes)}

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = os.path.join(self.img_dir, row['label'], row['image_id'])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.label2idx[row['label']]
        return image, label

In [4]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # normalize images to ImageNet mean and std
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
train_transform = transforms.Compose([
    # data augmentations
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [6]:
img_dir = "../data/train_images"
train_dataset = PaddyDataset(
    df=train_df,
    img_dir=img_dir,
    transformer=train_transform
)
val_dataset = PaddyDataset(
    df=val_df,
    img_dir=img_dir,
    transformer=val_transform
)

In [7]:
batch_size = 32
num_workers = multiprocessing.cpu_count()

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

In [8]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

In [9]:
resnet50.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [10]:
resnet50.fc.in_features

2048

In [11]:
num_classes = 10
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

In [12]:
model = resnet50
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
device

device(type='cuda')

In [13]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images , labels in train_loader:
        # move data to the right device
        images, labels = images.to(device), labels.to(device)
        
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds =torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    avg_loss = running_loss / total
    avg_acc = correct / total
    
    return avg_loss, avg_acc


In [14]:
def validate_one_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total
    avg_acc = correct / total
    return avg_loss, avg_acc


In [15]:
num_epochs = 3

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"  Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}")
    print(f"  Val Loss  : {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

Epoch 1/3
  Train loss: 0.8929 | Train acc: 0.7019
  Val Loss  : 5.1599 | Val   Acc: 0.2190
Epoch 2/3
  Train loss: 0.3691 | Train acc: 0.8828
  Val Loss  : 6.8450 | Val   Acc: 0.2056
Epoch 3/3
  Train loss: 0.2528 | Train acc: 0.9222
  Val Loss  : 6.7791 | Val   Acc: 0.2056


In [15]:
def train_model(train_loader, val_loader, device, num_epochs=10, lr=1e-4, num_classes=10):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    for param in model.parameters():
        param.requires_grad = False
    in_features = model.fc.in_features

    model.fc = nn.Linear(in_features, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)
    model = model.to(device)
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"  Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}")
        print(f"  Val Loss  : {val_loss:.4f}   | Val Acc  : {val_acc:.4f}")
    return model

In [17]:
model = train_model(train_loader, val_loader, device, num_epochs=3, lr=1e-4)

Epoch 1/3
  Train loss: 2.0420 | Train acc: 0.2898
  Val Loss  : 2.2891   | Val Acc  : 0.1715
Epoch 2/3
  Train loss: 1.8087 | Train acc: 0.4084
  Val Loss  : 2.3456   | Val Acc  : 0.1537
Epoch 3/3
  Train loss: 1.6723 | Train acc: 0.4625
  Val Loss  : 2.4373   | Val Acc  : 0.1422


In [18]:
model = train_model(train_loader, val_loader, device, num_epochs=3, lr=1e-3)

Epoch 1/3
  Train loss: 1.6797 | Train acc: 0.4302
  Val Loss  : 2.8395   | Val Acc  : 0.1830
Epoch 2/3
  Train loss: 1.3564 | Train acc: 0.5474
  Val Loss  : 3.0731   | Val Acc  : 0.2012
Epoch 3/3
  Train loss: 1.2673 | Train acc: 0.5720
  Val Loss  : 3.3108   | Val Acc  : 0.1888


In [21]:
def train_model(train_loader, val_loader, device, num_epochs=10, lr_fc=1e-4, lr_backbone=1e-5, weight_decay=1e-4, num_classes=10):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    for param in model.parameters():
        param.requires_grad = False
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    for param in model.layer4.parameters():
        param.requires_grad = True
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam([
        {"params": model.fc.parameters(), "lr": lr_fc},
        {"params": model.layer4.parameters(), "lr": lr_backbone}
    ], weight_decay=weight_decay)
    model = model.to(device)
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"  Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}")
        print(f"  Val Loss  : {val_loss:.4f}   | Val Acc  : {val_acc:.4f}")
    return model

In [ ]:
model = train_model(
    train_loader, 
    val_loader, 
    device, 
    num_epochs=5, 
    lr_fc=1e-4, 
    lr_backbone=1e-5,
    weight_decay=0
)

Epoch 1/5
  Train loss: 1.6815 | Train acc: 0.4404
  Val Loss  : 2.9722   | Val Acc  : 0.1907
Epoch 2/5
  Train loss: 1.0682 | Train acc: 0.6448
  Val Loss  : 3.8842   | Val Acc  : 0.2065
Epoch 3/5
  Train loss: 0.8128 | Train acc: 0.7396
  Val Loss  : 4.4757   | Val Acc  : 0.2080
Epoch 4/5
  Train loss: 0.6350 | Train acc: 0.7975
  Val Loss  : 4.9139   | Val Acc  : 0.2161
Epoch 5/5
  Train loss: 0.5127 | Train acc: 0.8407
  Val Loss  : 5.4215   | Val Acc  : 0.2104


In [22]:
model = train_model(
    train_loader, 
    val_loader, 
    device, 
    num_epochs=5, 
    lr_fc=1e-4, 
    lr_backbone=1e-5,
    weight_decay=1e-4
)

Epoch 1/5
  Train loss: 1.6994 | Train acc: 0.4371
  Val Loss  : 2.9490   | Val Acc  : 0.1926
Epoch 2/5
  Train loss: 1.0712 | Train acc: 0.6507
  Val Loss  : 3.9615   | Val Acc  : 0.1950
Epoch 3/5
  Train loss: 0.7998 | Train acc: 0.7443
  Val Loss  : 4.5405   | Val Acc  : 0.2128
Epoch 4/5
  Train loss: 0.6415 | Train acc: 0.7917
  Val Loss  : 5.1688   | Val Acc  : 0.2075
Epoch 5/5
  Train loss: 0.5276 | Train acc: 0.8326
  Val Loss  : 5.5829   | Val Acc  : 0.2118


In [23]:
train_transform = transforms.Compose([
    # data augmentations
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


train_dataset = PaddyDataset(
    df=train_df,
    img_dir=img_dir,
    transformer=train_transform
)

val_dataset = PaddyDataset(
    df=val_df,
    img_dir=img_dir,
    transformer=val_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

In [24]:
model = train_model(
    train_loader, 
    val_loader, 
    device, 
    num_epochs=5, 
    lr_fc=1e-4, 
    lr_backbone=1e-5,
    weight_decay=1e-4
)

Epoch 1/5
  Train loss: 1.8100 | Train acc: 0.3850
  Val Loss  : 2.7625   | Val Acc  : 0.1931
Epoch 2/5
  Train loss: 1.3408 | Train acc: 0.5471
  Val Loss  : 3.3028   | Val Acc  : 0.2037
Epoch 3/5
  Train loss: 1.1507 | Train acc: 0.6067
  Val Loss  : 3.8394   | Val Acc  : 0.2185
Epoch 4/5
  Train loss: 1.0239 | Train acc: 0.6567
  Val Loss  : 4.0123   | Val Acc  : 0.2224
Epoch 5/5
  Train loss: 0.9170 | Train acc: 0.6906
  Val Loss  : 4.3266   | Val Acc  : 0.2137
